In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True

# import packages
%matplotlib inline
import math
import sys
# sys.path.append('.')  # or full path to CompSep
np.math = math  # Redirect numpy.math to the built-in math module

import denoising
import importlib
denoising = importlib.reload(denoising)
import utils
utils = importlib.reload(utils)

In [3]:
signal_Q = np.load(f"/Users/tsouros/Desktop/Planck data/BK_CMB_S4_north_patch/signal/patch_166_Q353_PR4_no_PS_map_zero_adjust_cen_pix_lon_326.22802734375_lat_-41.81031489577862.npy")
signal_Q = utils.downsample_by_four(signal_Q)
signal_Q = signal_Q[None, :, :]

signal_U = np.load(f"/Users/tsouros/Desktop/Planck data/BK_CMB_S4_north_patch/signal/patch_166_U353_PR4_no_PS_map_zero_adjust_cen_pix_lon_326.22802734375_lat_-41.81031489577862.npy")
signal_U = utils.downsample_by_four(signal_U)
signal_U = signal_U[None, :, :]

In [4]:
import numpy as np
import glob

# Get sorted list of file paths
nuisance_Q = sorted(glob.glob("/Users/tsouros/Desktop/Planck data/BK_CMB_S4_north_patch/nuisance/patch_166_noise_Q353_PR4_no_PS_map_zero_adjust_cen_pix_lon_326.22802734375_lat_-41.81031489577862_noise_seed_*.npy"))
nuisance_U = sorted(glob.glob("/Users/tsouros/Desktop/Planck data/BK_CMB_S4_north_patch/nuisance/patch_166_noise_U353_PR4_no_PS_map_zero_adjust_cen_pix_lon_326.22802734375_lat_-41.81031489577862_noise_seed_*.npy"))

# Load all arrays and stack into shape (N_maps, 768, 768)
contamination_arr_Q = np.stack([utils.downsample_by_four(np.load(f))[None, :, :] for f in nuisance_Q], axis=0)
contamination_arr_U = np.stack([utils.downsample_by_four(np.load(f))[None, :, :] for f in nuisance_U], axis=0)

contamination_arr = np.stack([contamination_arr_Q, contamination_arr_U], axis=1)

In [7]:
data = (signal_Q, signal_U)

np.save('contamination_arr.npy', contamination_arr)
np.save('data.npy', data)

In [4]:
image_target = data
threshold_func = None
remove_edge = False
std = denoising.compute_std(image_target, contamination_arr = contamination_arr, s_cov_func = threshold_func, remove_edge=remove_edge, precision='double')
std_double = denoising.compute_std_double(image_target, contamination_arr = contamination_arr, remove_edge=remove_edge, precision='double')

In [5]:
image_init = image_target

n_epochs = 3 #number of epochs
# decontaminate
for i in range(n_epochs):
    print(f'Starting epoch {i+1}')
    running_map = denoising.denoise(image_target, contamination_arr = contamination_arr, std = std, std_double=std_double ,seed=0, print_each_step=True, steps = 25, n_batch = 25, s_cov_func=threshold_func, image_init = image_init, remove_edge=remove_edge, precision='double')
    running_map = (running_map[0], running_map[1])

    std = denoising.compute_std(running_map, contamination_arr = contamination_arr, remove_edge=remove_edge)
    std_double = denoising.compute_std_double(running_map, contamination_arr = contamination_arr, remove_edge=remove_edge)

image_syn_Q = running_map[0]
image_syn_U = running_map[1]

Starting epoch 1
# of estimators:  54827
Current Loss: 3.84e+01


KeyboardInterrupt: 

In [ ]:
# Convert tuples to NumPy arrays
data = np.stack([signal_Q[0], signal_U[0]])  # Shape: (2, ...)
image_denoised = np.stack([image_syn_Q[0], image_syn_U[0]])  # Ensure it's an array

results = np.array([data, image_denoised])
np.save(f"recon", results)